[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/openlayer-ai/examples-gallery/blob/main/tabular-classification/documentation-tutorial/tabular-tutorial-part-1.ipynb)

# <a id="top">Openlayer tabular tutorial - Part 1</a>

Welcome to the tabular tutorial notebook! You should use this notebook together with the **tabular tutorial from our documentation**.


## <a id="toc">Table of contents</a>

1. [**Getting the data and training the model**](#1)
    

2. [**Using Openlayer's Python API**](#2)

In [ ]:
%%bash

if [ ! -e "requirements.txt" ]; then
    curl "https://raw.githubusercontent.com/openlayer-ai/examples-gallery/main/tabular-classification/documentation-tutorial/requirements.txt" --output "requirements.txt"
fi

In [ ]:
!pip install -r requirements.txt

## <a id="1"> 1. Getting the data and training the model </a>

[Back to top](#top)

In this first part, we will get the dataset, pre-process it, split it into training and validation sets, and train a model. Feel free to skim through this section if you are already comfortable with how these steps look for an sklearn model.   

In [ ]:
import numpy as np
import pandas as pd

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

### <a id="download">Downloading the dataset </a>

We have stored the dataset on the following S3 bucket. If, for some reason, you get an error reading the csv directly from it, feel free to copy and paste the URL in your browser and download the csv file. The dataset we use is a modified version of the Churn Modeling dataset from [this Kaggle competition](https://www.kaggle.com/competitions/churn-modelling/overview).

In [ ]:
%%bash

if [ ! -e "churn_train.csv" ]; then
    curl "https://openlayer-static-assets.s3.us-west-2.amazonaws.com/examples-datasets/tabular-classification/documentation/churn_train.csv" --output "churn_train.csv"
fi

if [ ! -e "churn_val.csv" ]; then
    curl "https://openlayer-static-assets.s3.us-west-2.amazonaws.com/examples-datasets/tabular-classification/documentation/churn_val.csv" --output "churn_val.csv"
fi

In [ ]:
train_df = pd.read_csv("./churn_train.csv")
val_df = pd.read_csv("./churn_val.csv")

In [ ]:
train_df.head()

In [ ]:
feature_names = [
    "CreditScore", 
    "Geography",
    "Gender",
    "Age", 
    "Tenure",
    "Balance",
    "NumOfProducts",
    "HasCrCard",
    "IsActiveMember",
    "EstimatedSalary",
    "AggregateRate",
    "Year"
]
label_column_name = "Exited"

x_train = train_df[feature_names]
y_train = train_df[label_column_name]

x_val = val_df[feature_names]
y_val = val_df[label_column_name]

### <a id="prepare">Preparing the data</a>

In [ ]:
def data_encode_one_hot(df, encoders):
    """ Encodes categorical features using one-hot encoding. """
    df = df.copy(True)
    df.reset_index(drop=True, inplace=True) # Causes NaNs otherwise
    for feature, enc in encoders.items():
        enc_df = pd.DataFrame(enc.transform(df[[feature]]).toarray(), columns=enc.get_feature_names([feature]))
        df = df.join(enc_df)
        df = df.drop(columns=feature)
    return df

In [ ]:
def create_encoder_dict(df, categorical_feature_names):
    """ Creates encoders for each of the categorical features. 
        The predict function will need these encoders. 
    """
    from sklearn.preprocessing import OneHotEncoder
    encoders = {}
    for feature in categorical_feature_names:
        enc = OneHotEncoder(handle_unknown='ignore')
        enc.fit(df[[feature]])
        encoders[feature] = enc
    return encoders

In [ ]:
encoders = create_encoder_dict(x_train, ['Geography', 'Gender'])

In [ ]:
x_train_one_hot = data_encode_one_hot(x_train, encoders)
x_val_one_hot = data_encode_one_hot(x_val, encoders)

In [ ]:
# Imputation with the training set's mean to replace NaNs 
x_train_one_hot_imputed = x_train_one_hot.fillna(x_train_one_hot.mean(numeric_only=True))
x_val_one_hot_imputed = x_val_one_hot.fillna(x_train_one_hot.mean(numeric_only=True))

### <a id="train">Training the model</a>

In [ ]:
sklearn_model = GradientBoostingClassifier(random_state=1300)
sklearn_model.fit(x_train_one_hot_imputed, y_train)

In [ ]:
print(classification_report(y_val, sklearn_model.predict(x_val_one_hot_imputed)))

## <a id="2"> 2. Using Openlayer's Python API</a>

[Back to top](#top)

Now it's time to upload the datasets and model to the Openlayer platform.

In [ ]:
!pip install openlayer

### <a id="client">Instantiating the client</a>

In [ ]:
import openlayer

client = openlayer.OpenlayerClient("YOUR_API_KEY_HERE")

### <a id="project">Creating a project on the platform</a>

In [ ]:
from openlayer.tasks import TaskType

project = client.create_or_load_project(
    name="Churn Prediction",
    task_type=TaskType.TabularClassification,
    description="Evaluation of ML approaches to predict churn"
)

### <a id="dataset">Uploading datasets</a>

Before adding the datasets to a project, we need to do two things:
1. Enhance the dataset with additional columns to make it comprehensive, such as adding a column for labels and one for model predictions (if you're uploading a model as well).
2. Prepare a `dataset_config.yaml` file. This is a file that contains all the information needed by the Openlayer platform to utilize the dataset. It should include the column names, the class names, etc. For details on the fields of the `dataset_config.yaml` file, see the [API reference](https://reference.openlayer.com/reference/api/openlayer.OpenlayerClient.add_dataset.html#openlayer.OpenlayerClient.add_dataset).

Let's start by enhancing the datasets with the extra columns:

In [ ]:
# Adding the column with the labels
training_set = x_train.copy(deep=True)
training_set["Exited"] = y_train.values
validation_set = x_val.copy(deep=True)
validation_set["Exited"] = y_val.values

In [ ]:
# Adding the column with the predictions (since we'll also upload a model later)
training_set["predictions"] = sklearn_model.predict_proba(x_train_one_hot_imputed).tolist()
validation_set["predictions"] = sklearn_model.predict_proba(x_val_one_hot_imputed).tolist()

Now, we can prepare the `dataset_config.yaml` files for the training and validation sets.

In [ ]:
# Some variables that will go into the `dataset_config.yaml` file
categorical_feature_names = ["Gender", "Geography"]
class_names = ["Retained", "Exited"]
column_names = list(training_set.columns)
feature_names = list(x_val.columns)
label_column_name = "Exited"
predictions_column_name = "predictions"

In [ ]:
import yaml 

# Note the camelCase for the dict's keys
training_dataset_config = {
    "categoricalFeatureNames": categorical_feature_names,
    "classNames": class_names,
    "columnNames": column_names,
    "featureNames":feature_names,
    "label": "training",
    "labelColumnName": label_column_name,
    "predictionsColumnName": predictions_column_name,
}

with open("training_dataset_config.yaml", "w") as dataset_config_file:
    yaml.dump(training_dataset_config, dataset_config_file, default_flow_style=False)

In [ ]:
import copy

validation_dataset_config = copy.deepcopy(training_dataset_config)

# In our case, the only field that changes is the `label`, from "training" -> "validation"
validation_dataset_config["label"] = "validation"

with open("validation_dataset_config.yaml", "w") as dataset_config_file:
    yaml.dump(validation_dataset_config, dataset_config_file, default_flow_style=False)

In [ ]:
# Training set
project.add_dataframe(
    dataset_df=training_set,
    dataset_config_file_path="training_dataset_config.yaml",
)

In [ ]:
# Validation set
project.add_dataframe(
    dataset_df=validation_set,
    dataset_config_file_path="validation_dataset_config.yaml",
)

We can check that both datasets are now staged using the `project.status()` method. 

In [ ]:
project.status()

### <a id="model">Uploading models</a>

In this part of the tutorial, we will upload a **shell model**. Shell models are the most straightforward way to get started. They are comprised of metadata and all of the analysis are done via its predictions (which are [uploaded with the datasets](#dataset).)

To upload a shell model, we only need to define its name, the architecture type, and add some metadata that will be rendered in the platform to help us identify it. This information should be saved to a `model_config.yaml` file.

Let's create a `model_config.yaml` file for our model:

In [ ]:
import yaml

model_config = {
    "name": "Churn classifier",
    "architectureType": "sklearn",
    "metadata": {  # Can add anything here, as long as it is a dict
        "model_type": "Gradient Boosting Classifier",
        "regularization": "None",
        "encoder_used": "One Hot",
        "imputation": "Imputed with the training set's mean"
    },
    "classNames": class_names,
    "featureNames": feature_names,
    "categoricalFeatureNames": categorical_feature_names,
}

with open("model_config.yaml", "w") as model_config_file:
    yaml.dump(model_config, model_config_file, default_flow_style=False)

In [ ]:
project.add_model(
    model_config_file_path="model_config.yaml",
)

We can check that both datasets and model are staged using the `project.status()` method.

In [ ]:
project.status()

### <a id="commit"> Committing and pushing to the platform </a>

Finally, we can commit the first project version to the platform. 

In [ ]:
project.commit("Initial commit!")

In [ ]:
project.status()

In [ ]:
project.push()